In [1]:
import pandas as pd
import os
import logging
from typing import List, Dict, Any
from langchain_core.documents import Document

# 配置日志
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def build_document(
    row: pd.Series,
    sheet_name: str,
    filename: str,
    content_fields: List[str],
    metadata_fields: Dict[str, str]
) -> Document:
    """通用文档构建函数
    
    Args:
        row: 数据行
        sheet_name: 工作表名称
        filename: 文件名
        content_fields: 需要放入page_content的字段列表
        metadata_fields: 元数据字段映射 {元数据键名: 数据列名}
    
    Returns:
        LangChain Document对象
    """
    # 构建元数据
    metadata = {"来源表格": f"{filename}，{sheet_name}"}
    for meta_key, col_name in metadata_fields.items():
        metadata[meta_key] = str(row[col_name])  # 确保所有值为字符串
    
    # 构建内容
    content_parts = []
    for field in content_fields:
        content_parts.append(f"{field}：{str(row[field])}")
    
    return Document(
        page_content="\n".join(content_parts),
        metadata=metadata
    )

def validate_dataframe(df: pd.DataFrame, required_columns: List[str], sheet_name: str) -> None:
    """验证DataFrame是否包含必需列"""
    missing_cols = set(required_columns) - set(df.columns)
    if missing_cols:
        raise ValueError(
            f"工作表 '{sheet_name}' 缺少必要列: {missing_cols}\n"
            f"现有列: {list(df.columns)}"
        )

def process_excel_to_rag(sheet_configs:dict,file_path: str) -> List[Document]:
    """处理Excel文件生成RAG文档
    
    Args:
        file_path: Excel文件路径
    
    Returns:
        包含所有文档的列表
    """
    # 获取基础文件名（不带路径）
    filename = os.path.basename(file_path)
    documents = []
    # 处理每个工作表
    for sheet_name, config in sheet_configs.items():
        try:
            # 读取数据
            df = pd.read_excel(file_path, sheet_name=sheet_name)
            logger.info(f"成功读取工作表: {sheet_name}, 共 {len(df)} 行")
            
            # 数据验证
            validate_dataframe(df, config['required_columns'], sheet_name)
            
            # 处理每一行
            for _, row in df.iterrows():
                # 跳过关键字段为空的行
                #if pd.isnull(row['病历']):
                    #logger.warning(f"跳过空病历记录（工作表 {sheet_name}）")
                    #continue
                
                # 构建文档
                doc = build_document(
                    row=row,
                    sheet_name=sheet_name,
                    filename=filename,
                    content_fields=config['content_fields'],
                    metadata_fields=config['metadata_fields']
                )
                documents.append(doc)
                
        except Exception as e:
            logger.error(f"处理工作表 {sheet_name} 时出错: {str(e)}")
            raise
    
    logger.info(f"共生成 {len(documents)} 个文档")
    return documents
excel_path_0 = "A-2-5_模型喂养数据-护理病历质控规则.xlsx"
sheet_configs_0 = {
    '汇总': {
        'required_columns': ['性别','年龄','入院诊断','体征单诊断','病历','检查项目','备注','问题控件'],
        'content_fields': ['性别','年龄','入院诊断','体征单诊断','病历','检查项目','备注','问题控件'],
        'metadata_fields': {'性别': '性别', '年龄': '年龄', '入院诊断': '入院诊断', '体征单诊断': '体征单诊断', '病历': '病历', '检查项目': '检查项目', '备注': '备注', '问题控件': '问题控件'}
    }
}

excel_path_1 = "A-2-5-1_模型喂养数据-护理病历质控规则-文字版.xlsx"
sheet_configs_1 = {
    '质控规则': {
        'required_columns': ['项目', '项目检查内容释义','二级标题','三级标题'],
        'content_fields': ['项目', '项目检查内容释义','二级标题','三级标题'],
        'metadata_fields': {'项目': '项目', '项目检查内容释义': '项目检查内容释义', '二级标题': '二级标题', '三级标题': '三级标题'}
    },
    '质控问题': {
        'required_columns': ['质控问题'],
        'content_fields': ['质控问题'],
        'metadata_fields': {'质控问题': '质控问题'}
    },
    '记录单名称': {
        'required_columns': ['病历名称'],
        'content_fields': ['病历名称'],
        'metadata_fields': {
            '病历名称': '病历名称'
        }
    }
}

excel_path_2 = "A-2-5-2_模型喂养数据-护理病历质控规则-数据库配置版.xlsx"
sheet_configs_2 = {
    '时效性': {
        'required_columns': ['病历', '完成时间'],
        'content_fields': ['病历', '完成时间'],
        'metadata_fields': {'病历类型': '病历', '完成时间': '完成时间'}
    },
    '完整性': {
        'required_columns': ['病历', '必填项'],
        'content_fields': ['病历', '必填项'],
        'metadata_fields': {'病历类型': '病历', '必填项': '必填项'}
    },
    '合理性-互补词': {
        'required_columns': ['病历', '互补词条件', '互补控件'],
        'content_fields': ['病历', '互补词条件', '互补控件'],
        'metadata_fields': {
            '病历类型': '病历', 
            '互补词条件': '互补词条件',
            '互补控件': '互补控件'
        }
    },
    '环节质控检查项': {
        'required_columns': ['病历', '环节点', '分类', '检查项'],
        'content_fields': ['病历', '环节点', '分类', '检查项'],
        'metadata_fields': {
            '病历类型': '病历',
            '环节节点': '环节点',
            '分类': '分类',
            '检查项': '检查项'
        }
    }
}

try:
    rag_docs_0 = process_excel_to_rag(sheet_configs_0,excel_path_0)
    rag_docs_1 = process_excel_to_rag(sheet_configs_1,excel_path_1)
    rag_docs_2 = process_excel_to_rag(sheet_configs_2,excel_path_2)
except Exception as e:
    logger.error(f"处理Excel文件失败: {str(e)}")
    raise

INFO:__main__:成功读取工作表: 汇总, 共 24380 行
INFO:__main__:共生成 24380 个文档
INFO:__main__:成功读取工作表: 质控规则, 共 316 行
INFO:__main__:成功读取工作表: 质控问题, 共 144 行
INFO:__main__:成功读取工作表: 记录单名称, 共 42 行
INFO:__main__:共生成 502 个文档
INFO:__main__:成功读取工作表: 时效性, 共 6 行
INFO:__main__:成功读取工作表: 完整性, 共 130 行
INFO:__main__:成功读取工作表: 合理性-互补词, 共 65 行
INFO:__main__:成功读取工作表: 环节质控检查项, 共 314 行
INFO:__main__:共生成 515 个文档


In [2]:
rag_docs_2

[Document(metadata={'来源表格': 'A-2-5-2_模型喂养数据-护理病历质控规则-数据库配置版.xlsx，时效性', '病历类型': '护理记录单', '完成时间': '入院后8小时完成'}, page_content='病历：护理记录单\n完成时间：入院后8小时完成'),
 Document(metadata={'来源表格': 'A-2-5-2_模型喂养数据-护理病历质控规则-数据库配置版.xlsx，时效性', '病历类型': '入院评估单', '完成时间': '入院后8小时完成'}, page_content='病历：入院评估单\n完成时间：入院后8小时完成'),
 Document(metadata={'来源表格': 'A-2-5-2_模型喂养数据-护理病历质控规则-数据库配置版.xlsx，时效性', '病历类型': '住院患者/患儿压力性损伤评估及措施单', '完成时间': '入院后8小时完成'}, page_content='病历：住院患者/患儿压力性损伤评估及措施单\n完成时间：入院后8小时完成'),
 Document(metadata={'来源表格': 'A-2-5-2_模型喂养数据-护理病历质控规则-数据库配置版.xlsx，时效性', '病历类型': '日常生活能力评估及措施单', '完成时间': '入院后8小时完成'}, page_content='病历：日常生活能力评估及措施单\n完成时间：入院后8小时完成'),
 Document(metadata={'来源表格': 'A-2-5-2_模型喂养数据-护理病历质控规则-数据库配置版.xlsx，时效性', '病历类型': '深静脉血栓评估及措施单', '完成时间': '入院后8小时完成'}, page_content='病历：深静脉血栓评估及措施单\n完成时间：入院后8小时完成'),
 Document(metadata={'来源表格': 'A-2-5-2_模型喂养数据-护理病历质控规则-数据库配置版.xlsx，时效性', '病历类型': '住院患者/患儿跌倒坠床评估及措施单', '完成时间': '入院后8小时完成'}, page_content='病历：住院患者/患儿跌倒坠床评估及措施单\n完成时间：入院后8小时完成'),
 Document(metadata={'来源表

In [3]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.vectorstores import Chroma

def vectorize_documents(
    all_docs: List[Document],
    embedding_model: str = "local",  # 可选 "openai"
    vector_db: str = "faiss",        # 可选 "chroma"
    chunk_size: int = 1000,
    chunk_overlap: int = 200
) -> Any:  # 返回向量库对象
    """文档向量化处理"""
    
    # 合并所有文档
    combined_docs = [doc for docs_group in [rag_docs_0, rag_docs_1, rag_docs_2] 
                    for doc in docs_group]
    
    # 文本分割
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
        is_separator_regex=False,
    )
    splits = text_splitter.split_documents(combined_docs)
    
    # 初始化嵌入模型
    if embedding_model == "local":
        # 使用本地HuggingFace模型
        embeddings = HuggingFaceEmbeddings(
            model_name="GanymedeNil/text2vec-base-chinese",
            model_kwargs={'device': 'cpu'},  # 使用GPU可改为 'cuda'
            encode_kwargs={'normalize_embeddings': True}
        )
    elif embedding_model == "openai":
        # 需要设置环境变量 OPENAI_API_KEY
        embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
    
    # 创建向量库
    if vector_db == "faiss":
        return FAISS.from_documents(splits, embeddings)
    elif vector_db == "chroma":
        return Chroma.from_documents(splits, embeddings)
    
    raise ValueError("不支持的向量数据库类型")

# 使用示例
vector_store = vectorize_documents(
    all_docs=[rag_docs_0, rag_docs_1, rag_docs_2],
    embedding_model="local",  # 切换模型时修改此参数
    vector_db="faiss",
    chunk_size=1000,
    chunk_overlap=200
)

ModuleNotFoundError: No module named 'langchain_community'